In [1]:
%load_ext autoreload
%autoreload 2
%pylab inline

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [2]:
import sys
import glob
import pandas as pd
import os
import seaborn as sns
from tqdm import tqdm_notebook as tqdm
from statsmodels.distributions.empirical_distribution import ECDF
from collections import defaultdict
import open_spiel.python.examples.ubc_dispatch as dispatch
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
from sklearn.model_selection import ParameterGrid
from pathlib import Path
import torch.nn.functional as F
import torch

In [3]:
game_dir = os.environ["CLOCK_AUCTION_CONFIG_DIR"]

In [4]:
PREFIX = 'jun24'
sats_games = [f'{PREFIX}/' + os.path.basename(x).replace('.json','') for x in glob.glob(f'{game_dir}/{PREFIX}/*')]
sats_games

['jun24/jun24_0_base',
 'jun24/jun24_3_medium_speed',
 'jun24/jun24_1_undersell_allowed',
 'jun24/jun24_1_medium_speed',
 'jun24/jun24_1_base',
 'jun24/jun24_4_base',
 'jun24/jun24_3_undersell_allowed',
 'jun24/jun24_1_no_activity',
 'jun24/jun24_2_high_speed',
 'jun24/jun24_4_no_activity',
 'jun24/jun24_4_medium_speed',
 'jun24/jun24_0_medium_speed',
 'jun24/jun24_3_base',
 'jun24/jun24_2_base',
 'jun24/jun24_3_no_activity',
 'jun24/jun24_2_medium_speed',
 'jun24/jun24_0_no_activity',
 'jun24/jun24_0_high_speed',
 'jun24/jun24_1_high_speed',
 'jun24/jun24_4_high_speed',
 'jun24/jun24_4_undersell_allowed',
 'jun24/jun24_2_undersell_allowed',
 'jun24/jun24_2_no_activity',
 'jun24/jun24_3_high_speed',
 'jun24/jun24_0_undersell_allowed']

# For running the real dumb test case

In [ ]:
pgrid = ParameterGrid([
    {
        # Defaults
        'game': [g for g in sats_games]# if 'hide' not in g and 'imperfect' not in g]
    },
])
print(len(pgrid))

25


In [12]:
num_iters = 999_000_000
time_limit_seconds = int(3600 * 0.5)
for config in pgrid:
    CONFIG = 'cfr_gamut' # The YML network config folder for the main jobs
    N_SEEDS = 1 # Number of seeds of each experiment to launch
    extra_name = ''
    overrides = f'--overwrite_db True --eval_every 999_999_000 --eval_zero false --total_timesteps {num_iters} --dispatch_br false --report_freq 5000 --time_limit_seconds {time_limit_seconds}'      
    dispatch.dispatch_experiments(CONFIG, base_job_name='jun24_cfr_gamut_2', game_name=config['game'], overrides=overrides, n_seeds=N_SEEDS, extra_name=extra_name, alg='cfr')

Submitted batch job 87351
Submitted batch job 87352
Submitted batch job 87353
Submitted batch job 87354
Submitted batch job 87355
Submitted batch job 87356
Submitted batch job 87357
Submitted batch job 87358
Submitted batch job 87359
Submitted batch job 87360
Submitted batch job 87361
Submitted batch job 87362
Submitted batch job 87363
Submitted batch job 87364
Submitted batch job 87365
Submitted batch job 87366
Submitted batch job 87367
Submitted batch job 87368
Submitted batch job 87369
Submitted batch job 87370
Submitted batch job 87371
Submitted batch job 87372
Submitted batch job 87373
Submitted batch job 87374
Submitted batch job 87375
Submitted batch job 87376
Submitted batch job 87377
Submitted batch job 87378
Submitted batch job 87379
Submitted batch job 87380
Submitted batch job 87381
Submitted batch job 87382
Submitted batch job 87383
Submitted batch job 87384
Submitted batch job 87385
Submitted batch job 87386
Submitted batch job 87387
Submitted batch job 87388
Submitted ba

In [ ]:
num_iters = 10_000_000
time_limit_seconds = 3600 * 4
for config in pgrid:
    CONFIG = 'cfr_external' # The YML network config folder for the main jobs
    N_SEEDS = 3 # Number of seeds of each experiment to launch
    extra_name = ''
    overrides = f'--overwrite_db True --eval_every 250_000 --eval_zero false --total_timesteps {num_iters} --dispatch_br false --report_freq 5000 --time_limit_seconds {time_limit_seconds}'      
    dispatch.dispatch_experiments(CONFIG, base_job_name='jun23external', game_name=config['game'], overrides=overrides, n_seeds=N_SEEDS, extra_name=extra_name, alg='cfr')

In [ ]:
num_iters = 30_000_000
time_limit_seconds = 3600 * 4
for config in pgrid:
    CONFIG = 'cfr_outcome' # The YML network config folder for the main jobs
    N_SEEDS = 3 # Number of seeds of each experiment to launch
    extra_name = ''
    overrides = f'--overwrite_db True --eval_every 999_000_000 --eval_zero false --total_timesteps {num_iters} --dispatch_br false --report_freq 5000 --time_limit_seconds {time_limit_seconds}'      
    dispatch.dispatch_experiments(CONFIG, base_job_name='jun23outcome_2', game_name=config['game'], overrides=overrides, n_seeds=N_SEEDS, extra_name=extra_name, alg='cfr')

In [ ]:
# for config in pgrid:
#     CONFIG = 'ppo_no_ent_jun14'
#     # CONFIG = 'ppo_may31_23' # The YML network config folder for the main jobs
#     # BR_CONFIG = 'ppo_may31_23' 
#     N_SEEDS = 3 # Number of seeds of each experiment to launch
#     extra_name = ''
#     overrides = f'--overwrite_db True --br_portfolio_path {BR_CONFIG} --eval_every 999_000_000 --eval_zero false --total_timesteps 5_000_000 --dispatch_br false --br_overrides "--total_timesteps 1_000_000"'        
#     dispatch.dispatch_experiments(CONFIG, base_job_name='jun14ent_3', game_name=config['game'], overrides=overrides, n_seeds=N_SEEDS, extra_name=extra_name)

In [15]:
!squeue --format="%.18i %.90j %.2t .%.10M %.6D %R"

             JOBID                                                                                       NAME ST .      TIME  NODES NODELIST(REASON)
             87464                          jun24_jun24_3_base-cfr_gamutoutcome_lin_avg-100_jun24_cfr_gamut_2 PD .      0:00      1 (Resources)
             87465                     jun24_jun24_3_base-cfr_gamutoutcome_explore_more-100_jun24_cfr_gamut_2 PD .      0:00      1 (Priority)
             87466                                  jun24_jun24_3_base-cfr_gamutoutcome-100_jun24_cfr_gamut_2 PD .      0:00      1 (Priority)
             87467                         jun24_jun24_3_base-cfr_gamutexternal_lin_avg-100_jun24_cfr_gamut_2 PD .      0:00      1 (Priority)
             87468                                 jun24_jun24_2_base-cfr_gamutexternal-100_jun24_cfr_gamut_2 PD .      0:00      1 (Priority)
             87469                     jun24_jun24_2_base-cfr_gamutoutcome_lin_avg_plus-100_jun24_cfr_gamut_2 PD .      0:00      1 (Pr

In [ ]:
def check_queue():
    !ssh borg.cs.ubc.ca '/opt/slurm/bin/squeue -u newmanne --format="%.18i %.90j %.2t .%.10M %.6D %R"'


In [ ]:
!squeue --format "%.18i %.90j %.2t .%.10M %.6D %R"

In [ ]:
check_queue()

In [9]:
!squeue | wc -l

162


In [ ]:
!squeue | grep "may30" | wc -l

In [ ]:
!scancel -u ubuntu

In [ ]:
# !scancel  46891 46893

In [ ]:
!rm -rf /shared/outputs/minitest/
try:
    Experiment.objects.get(name='minitest').delete()
except:
    pass
dispatch.dispatch_experiments('feb2', base_job_name='minitest', game_name='tiny', overrides = '--br_portfolio_path feb2 --eval_every 500 --num_training_episodes 1_000 --br_overrides "--num_training_episodes 1_000" --eval_overrides "--num_samples 100"')

In [ ]:
BestResponse.objects.last()

In [ ]:
Experiment.objects.all()

In [ ]:
!sacct --starttime 2022-03-11 --format=User,JobID,Jobname,partition,state,time,start,end,elapsed,MaxRss,MaxVMSize,nnodes,ncpus,nodelist

In [ ]:
Evaluation.objects.last()

In [ ]:
os.system("ssh borg.cs.ubc.ca '/opt/slurm/bin/sbatch'")

In [16]:
!squeue --format="%.18i %.65j %.2t %.10M %.6D %R" | grep "gamut" | awk '{print $1}' | xargs scancel